# 02. 하둡이해하기

## 02-01 하둡이란

- 대용량의 데이터를 분산처리하게 해주는 아파치 재단의 오픈소스 프로젝트
- http://hadoop.apache.org
- 구글의 "The Google File System" 과 "MapReduce:Simplified Data Processing on Large Cluster" 라는 논문이 기원
- Nutch/Lucene 이라는 검색엔진 오픈소스를 만들고 있었던, 커그커팅이  검색엔진을 만들면서 발생한 문제를 해결하기 위해서 위의 2개의 논문을 적용함.
- hadoop이란 이름은 더그커팅의 아들이 갖고 놀던 노란색 코끼리 인형의 이름

### 하둡의 구성
- 하둡은  분산파일시스템인 HDFS( Hadoop Distributed File System)과 MapReduce라는 분산처리 시스템으로 구성됨.
- HDFS와 MapReduce은 물리적으로 하나의 서버에 같이 설치됨.

### 하둡의 특징
- 오픈소스
- 데이터가 있는 곳으로 코드를 이동
- 스케일 아웃 => 저렴한 서버를 이용해서 저장 용량과 처리 능력을 높임.
- 병렬처리를 가능하게 해주는 단순한 데이터 모델 : MapReduce의 특징이며,  프로그램이 단순하지만, 복잡한 알고리즘은 구현이 불가능한 것도 있음.
    - 구글은 70% MR로 처리하고 나머지는 30% MPI방식으로 처리함.
- 오프라인 배치 프로세싱에 최적화
    - 실시간 서비스에는 부적합.
    - 실시간 서비스에는 다른 오픈소스를 활용함. HBASE 또는 Cassandra 또는 MongoDB

## 02-02 하둡 아키텍처

- 현재 하둡은 2.X 버전이 나와 있지만, 하둡 1.X 기준으로 설명
- 하둡 1.X버전을 먼저 이해하고, 하둡 2.X를 이해하는 것이 더 쉬움.
- HDFS는 하나의 Namenode와 다수의 Datanode을 구성,   Namenode의 데이터를 주기적으로 백업하는 Secondary Namenode로 구성함.
- MapReduce는 하나의 Job Tracker와 다수의 Task Tracker로 구성.
![이미지](02_하둡구성도.png)

- **하둡 1.X의 문제점이 무엇인가 고민해보자.**
- 하둡 1.X의 문제점을 해결하고자 하둡 2.X가 나옴.

## 02-03 HFDS 개요

### HDFS의 특징
- 파일을 여러개의 블록으로 나누어 저장
   - 기본 64MB,  주로 128MB사용
   - 작은 크기의 파일을 다수 저장 용도는 부적합하고, 아주 큰 파일을 저장할때 유용
- 하부 운영체제의 파일 시스템을 그대로 사용
- 하드웨어가 고장나도 바로 시스템을 다운되지 않음.
    - 데이터 복제( Replication )
- Write Once Read Many : 한번 데이터 생성되면, 수정은 불가능하고 데이터 끝에 추가만 가능함.

![이미지](02_HDFS구조.jpg)

### 네임노드( Namenode )
- 네임노드 메타 정보
    - 파일에 대한 이름과 디렉토리 정보
    - 파일의 복제본 개수
    - 파일을 구성하는 데이터 블록ID 리스트와 블록별 저장된 데이터노드의 위치
    
- 네임노드의 문제점들 
    - 네임노드가 죽으면, 파일을 찾을 수가 없음.
    - 네임노드의 메타 정보가 손상받으면 해당 파일을 접근불가능.
    - 네임노드는 SPOF( Single Point Of Failure,  단일 장애점 )
    
- 2차 네임노드( Secondary Namenode )
    - 네임노드의 메타 정보를 주기적으로 백업
    - 주기적으로 백업한 체크포인트 정보와  에디토로그( Edit Log )을 병합하여 FsImage를 생성함.
    - FsImage가 체크포인트 정보가 됨.
    - 네임노드의 메타 정보가 손상되었을때,  FsImage을 이용해서 복구가 가능함.
    - 문제점 : 실시간 복구가 되지 않고 복구 작업이 오래 걸림.  사람이 수동작업이 필요함.
    
- 네임노드와 데이터노드간의 통신 : 하트비트( Heartbeat )
    - 네임노드가 데이터노드가 죽었는지 살았는지 파악함.
    - 데이터노드가 현재 상태를 네임노드에게 보고함.
    
### HDFS I/O설명
![이미지](02_HDFS_IO.png)

## 02-04 MapReduce 프레임워크 
![이미지](02_MapReduce.png)


### MapReduce 프레임워크의 특징
- 데이터가 있는 서버로 코드를 전송
- 데이터를 키/밸류 데이터셋의 변환으로 처리
- Share Nothing 아키텍처
    - 각각의 작업에 대해서 의존성이 없음.
    - 이 특징으로 어떤 작업은 구현이 더 어려움.
- 오프라인 배치처리에 적합

### 잡 트래커( Job Tracker )
-  잡( Job )과 태스트( Task ) 
    - 하나의 MapReduce 프로그램을 실행하면 잡( Job )이라고 함.
    - Job은 하나의 이상의 Map Task와 하나 이상의 Reduce로 구성함.
- 잡 트래커의 역할
    - 사용자로부터 MapReduce실행요청(  MR코드를포함한 Jar,  입력데이터 위치, 출력데이터 위치 등)을 받아서 TaskTacker들에게 나누어 주고, 실행이 종료될때까지 관리 역할
    - 하나의 Task가 실패하면 다른 TaskTracker에게 재실행 요청을 함.

![이미지](02_jobtracker.png)

### 잡 트래커와 태스트 트래커 간의 통신 : 하트비트( Heartbeat )
- 잡 트래커는 태스트 트래커로부터 주기적으로 현재 상태를 보고 받음.
    - 살아있는지 여부
    - 실행중인 테스트들의 상태
    - 태스트 트래커내에 있는 슬롯의 개수
  
### 잡 스케줄러
- 잡 스케줄러는 클러스터내에 여러 Job을 동시에 실행하도록 함.
- 설정 파일 수정으로 스케줄러를 변경 가능함.
- FIFO 스케줄러
     - 디폴트 스케줄러로 먼저 요청된 순서대로 실행
     - 5개의 우선순위가 있으면,  우선순위가 높은것을 먼저 실행함. 
     - 이미 실행중인 Job이 끝나기 전에는 다른 우선순위가 높은 Job이라도 실행되지 않음.
- 야후의 Capacity 스케줄러
    - 여러 개의 큐가 정의되고, 각 큐마다 클러스터내에서 사용할 수 있는 용량이 지정됨.
    - Job은 하나의 큐에 들어가고, 각 규 내에서는 FIFO스케줄러로 동작
    - 여러 부서 및 조직에서 하나의 거대한 하둡 클러스터를 사용할때 유용.
    
### MapReduce 프로그래밍
- 자바언어
- 스트리밍 방식의 언어들
    - 파이션,  펄, 루비,  R
- 파이프 방식  : C++
- Hive / Pig 등의 하이레벌 언어

## 02-05 MapReduce 사용 적합 분야와 부적합 분야

### 적합한 분야들
- 병렬도가 광장히 높은 단순 작업
- 로그 분석
- 머신러닝  : k-mean 군집 알고리즘

### 부적합 분야들
- 리얼타임 데이터 스트림처리
- 반복실행이 많이 필요한 작업들
    - Job을 실행하기 위한 준비단계가 좀 시간이 필요함.
    - Share Nothing 아키텍처를 가지고 있어서 Task간의 데이터 공유가 안됨.
    - 재귀적인 성격의 그래프 관련 알고리즘 => 그래프관련 오픈소스가 따로 존재함.
- MapReduce를 구현할때 네트워크 데이터 전송량이 너무 큰 경우

## 02-06 숙제
- 지금까지 나온 hadoop 1.X 의 문제점들이 무엇이 있는지 생각해보자.
